In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

In [ ]:
from genpen.genpen import *

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

# start with an interesting pattern?

In [ ]:
polys = MultiPolygon(so.triangulate(points, tolerance=0.5))
polys

## do the shading

In [ ]:
class Fill(Shape):
    
    def __init__(self):
        self._p = p
        
    def _repr_svg_(self):
        return self.fill._repr_svg_()
    
    @property
    def boundary(self):
        return self._p.boundary
    
    @property
    def fill(self):
        return self.fill_poly(self._p)
    
    @property
    def _all_geoms(self):
        return sg.GeometryCollection([self.boundary, self.fill])

class Angle(object):
    
    @property
    def deg(self):
        return self._deg
    
    @deg.setter
    def deg(self, deg):
        self._deg = deg
        self._rad = np.deg2rad(deg)
    
    @property
    def rad(self):
        return self._rad
    
    @rad.setter
    def rad(self, rad):
        self._deg = np.rad2deg(rad)
        self._rad = rad
        
    
    def __init__(self, deg=None, rad=None):
        if (deg is not None) and (rad is not None):
            print('WARNING: arguments entered for both deg and rad; defaulting to using deg')
        if deg is not None:
            self.deg = deg
        elif rad is not None:
            self.rad = rad
            
            
    def __repr__(self):
        return f'deg = {self.deg} \nrad = {self.rad:.3}'

@dataclass
class Filler(DataClassBase):
        
    def fill_poly(self, poly):
        return self.fill_func(poly)
    
    
    
@dataclass  
class HatchFill(Shape):
    
    def __init__(
        self,
        poly_to_fill:sg.Polygon,
        degrees:float=0.,
        spacing:float=0.5,  # usually mm
        alternate_direction:bool=True,
        fill_inscribe_buffer=1.1,
    ):
        self._ptf = poly_to_fill
        self.degrees = degrees
        self.spacing = spacing
        self.alternate_direction = alternate_direction
        self.fill_inscribe_buffer = fill_inscribe_buffer
    
    @property
    def angle(self):
        return Angle(deg=self.degrees)
    
    @property
    def inscribe_radius(self):
        d_from_furthest_vertex_to_centroid = self._ptf.hausdorff_distance(self._ptf.centroid)
        return d_from_furthest_vertex_to_centroid * self.fill_inscribe_buffer
    
    @property
    def inscribe_diameter(self):
        return self.inscribe_radius * 2
    
    @property
    def envelope_inscribe(self):
        return self._ptf.centroid.buffer(self.inscribe_radius)
    
    @property
    def envelope(self):
        # rotating geom around centroid will always be inside this
        return Shape(box(*self.envelope_inscribe.bounds))
    
    @property
    def _e(self):
        return self.envelope
    
    @property
    def spacings(self):
        try:
            len(self.spacing) > 1
            spacings = self.spacing
        except TypeError:
            # if spacing is a number, make list of uniform spacings
            spacings = np.arange(0, self.inscribe_diameter, self.spacing)
        return spacings
    
    @property
    def lines(self):
        left = self.envelope.left
        right = self.envelope.right
        top = self.envelope.top
        lines = []
        for ii, _spacing in enumerate(spacings):
            y = top - _spacing
            line = LineString(((left, y), (right, y)))
            lines.append(line)
            
        if self.alternate_direction:
            for i in range(len(lines)):
                if i % 2:
                    lines[i] = reverse_LineString(lines[i])
        
        return sg.MultiLineString(lines)
    
    @property
    def rotated_lines(self):
        return sa.rotate(self.lines, self.angle.deg, origin=self._ptf.centroid, use_radians=False)
    
    @property
    def fill(self):
        return self._ptf.intersection(self.rotated_lines)
    
    @property
    def _p(self):
        return self.fill
        
    
    

@dataclass  
class BezierHatchFill(HatchFill):
    
    def __init__(
        self,
        poly_to_fill:sg.Polygon,
        xjitter_func,
        yjitter_func,
        degrees:float=0.,
        spacing:float=0.5,  # usually mm
        alternate_direction:bool=True,
        fill_inscribe_buffer=1.01,
        n_nodes_per_line=20,
        n_eval_points=100,
        
    ):
        self._ptf = poly_to_fill
        self.degrees = degrees
        self.spacing = spacing
        self.alternate_direction = alternate_direction
        self.fill_inscribe_buffer = fill_inscribe_buffer
        self.n_nodes_per_line = n_nodes_per_line
        self.init_nodes = None
        self.xjitter_func = make_callable(xjitter_func)
        self.yjitter_func = make_callable(yjitter_func)
        self.n_eval_points = n_eval_points
    
    def initialize_nodes(self):
        # here to be expanded
        self.init_node_xs = np.linspace(self._e.left, self._e.right, self.n_nodes_per_line)
        self.init_node_ys = np.zeros_like(self.init_node_xs)
        self.init_nodes = np.stack([self.init_node_xs, self.init_node_ys])
    
    @property
    def node_sets(self):
        if self.init_nodes is None:
            self.initialize_nodes()
        xs = self.init_node_xs.copy()
        ys = self.init_node_ys.copy()
        nodes = np.stack([xs, ys])
        node_sets = [nodes.copy()]
        for ii in range(1, len(self.spacings)):
            xs += self.xjitter_func(xs.shape)
            ys += self.yjitter_func(ys.shape)
            nodes = np.stack([xs, ys])
            node_sets.append(nodes)
        return node_sets
        
    @property
    def ft_node_sets(self):
        return [np.asfortranarray(n) for n in self.node_sets]
    
    @property
    def curves(self):
        curves = []
        for node_set in self.ft_node_sets:
            curve = bezier.Curve(node_set, degree=(node_set.shape[1]-1))
            curves.append(curve)
        return curves
    
    @property
    def _interpolated_curves(self):
        i_curves = []
        for curve in self.curves:
            eval_points = np.linspace(0, 1, self.n_eval_points)
            x, y = curve.evaluate_multi(eval_points)
            i_curves.append(np.stack([x, y]).T)
        return i_curves
    
    @property
    def lines(self):
        lines = []
        
        for ii, curve in enumerate(self._interpolated_curves):
            spacing = self.spacings[ii]
            y = self._e.top - spacing
            _curve = curve.copy()
            _curve[:, 1] += y
            line = LineString(_curve)
            lines.append(line)
            
        if self.alternate_direction:
            for i in range(len(lines)):
                if i % 2:
                    lines[i] = reverse_LineString(lines[i])
        
        lines = [l for l in lines if l.length > 0]
        
        return sg.MultiLineString(lines)
    
    

In [ ]:
bhf = BezierHatchFill(
    spacing=1.,
    poly_to_fill=poly_to_fill, 
    xjitter_func=xjitter_func, 
    yjitter_func=yjitter_func)

In [ ]:
n_layers = 1

In [ ]:
layers = []
for ii in range(n_layers):
    fills = []
    for p in polys:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(1., 3.5)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.4, 1.5),
            degrees=np.random.uniform(0,180),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func)
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype(f'linemerge --tolerance 0.3mm linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/262_bez_hatch.svg')

# with subdivide

In [ ]:
def random_line_subdivide(poly, x0=None, x1=None):
    if x0 is None:
        x0 = np.random.uniform(0,1)
    if x1 is None:
        x1 = (x0 + 0.5) % 1
    return LineString([poly.boundary.interpolate(x, normalized=True) for x in [x0, x1]])

def random_bezier_subdivide(poly, x0=None, x1=None, n_eval_points=50):
    if x0 is None:
        x0 = np.random.uniform(0.2, 0.4)
    if x1 is None:
        x1 = np.random.uniform(0.6, 0.8)
    line = np.asfortranarray(random_line_subdivide(poly, x0, x1))
    bez_array = np.stack([line[0], poly.centroid, line[1]]).T
    curve1 = bezier.Curve(bez_array, degree=2)
    bez = curve1.evaluate_multi(np.linspace(0., 1., n_eval_points))
    return sg.asLineString(bez.T)

def split_along_longest_side_of_min_rectangle(poly, xgen=None):
    if xgen is None:
        xgen = gp.make_callable(0.5)
    mrrc = poly.minimum_rotated_rectangle.boundary.coords
    sides = [LineString([mrrc[i], mrrc[i+1]]) for i in range(4)]
    longest_sides = [sides[i] for i in np.argsort([-l.length for l in sides])[:2]]
    bps = [ls.interpolate(xgen(), normalized=True)for ls in longest_sides]

    return LineString([so.nearest_points(bp, poly.boundary)[1] for bp in bps])

def recursive_split(poly, split_func=random_line_subdivide, p_continue=0.7, depth=0, depth_limit=15, buffer_kwargs=None):
    
    if buffer_kwargs is None:
        buffer_kwargs = {'distance':0}
    polys = list(poly.difference(split_func(poly).buffer(1e-6)))
    split_polys = []
    
    for i, p in enumerate(polys):
        continue_draw = np.random.binomial(n=1, p=p_continue)
        
        if continue_draw and (depth<depth_limit):
            
            split_polys += recursive_split(
                p, split_func=split_func, p_continue=p_continue, 
                depth=depth+1, depth_limit=depth_limit,
                buffer_kwargs=buffer_kwargs
            ) 
        else:
            split_polys.append(p.buffer(**buffer_kwargs))
    return split_polys

def recursive_split_frac_buffer(poly, split_func=random_line_subdivide, p_continue=0.7, depth=0, depth_limit=15, buffer_kwargs=None, buffer_frac=-0.1):
    try:
        if buffer_kwargs is None:
            buffer_kwargs = {'join_style':2, 'cap_style':2}
        polys = list(poly.difference(split_func(poly).buffer(1e-6)))
        split_polys = []

        for i, p in enumerate(polys):
            continue_draw = np.random.binomial(n=1, p=p_continue)
            distance=p.centroid.distance(p.boundary)*buffer_frac
            bp = p.buffer(distance=distance, **buffer_kwargs)
            if continue_draw and (depth<depth_limit):

                split_polys += recursive_split_frac_buffer(
                    bp, split_func=split_func, p_continue=p_continue, 
                    depth=depth+1, depth_limit=depth_limit,
                    buffer_kwargs=buffer_kwargs, buffer_frac=buffer_frac
                ) 
            else:

                split_polys.append(bp)
        return split_polys
    except:
        return [poly]

In [ ]:
xgen = ss.uniform(loc=0.4, scale=0.02).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=10,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])


sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:
layers = []
for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 3.)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.8),
            degrees=np.random.uniform(0,180),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=50,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
for tolerance in [0.3, 0.6, 1.2]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
split_func = functools.partial(random_bezier_subdivide, x0=0.1, x1=0.9, n_eval_points=20)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func, 
    p_continue=0.9, 
    depth=0, 
    depth_limit=8,
    buffer_frac=-0.0
)


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 2

inds = np.arange(len(bps))
split_bps = np.array_split(bps, n_layers)

layers = []
for ii in range(n_layers):
    this_bps = split_bps[ii]
    fills = []
    for p in this_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 3.)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 1.),
            degrees=np.random.uniform(0,180),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify --tolerance 0.1')
for tolerance in [0.1, 0.2, 0.3, 0.6,]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/268_subdivide_bez_hatch.svg')

In [ ]:
# make page
paper_size = '6x6 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
circ = drawbox.centroid.buffer(55)

In [ ]:
xgen = ss.uniform(loc=0.3, scale=0.02).rvs
split_func = functools.partial(random_bezier_subdivide, x0=0.3, x1=0.9, n_eval_points=20)
splits = recursive_split_frac_buffer(
    circ, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=3,
    buffer_frac=-0.0
)


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 2

inds = np.arange(len(bps))
split_bps = np.array_split(bps, n_layers)

layers = []
for ii in range(n_layers):
    this_bps = split_bps[ii]
    fills = []
    for p in this_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 1.2)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.6),
            degrees=np.random.uniform(-90,90),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify --tolerance 0.1')
for tolerance in [0.1, 0.2, 0.3,]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/272_subdivide_bez_hatch.svg')

In [ ]:
# make page
paper_size = '6x6 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
circ = drawbox.centroid.buffer(55)

In [ ]:
xgen = ss.uniform(loc=0.3, scale=0.02).rvs
split_func = functools.partial(random_bezier_subdivide, x0=0.3, x1=0.8, n_eval_points=20)
splits = recursive_split_frac_buffer(
    circ, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=5,
    buffer_frac=-0.0
)


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

inds = np.arange(len(bps))
split_bps = np.array_split(bps, n_layers)

layers = []
for ii in range(n_layers):
    this_bps = split_bps[ii]
    fills = []
    for p in this_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 1.2)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.6),
            degrees=np.random.uniform(-90,90),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify --tolerance 0.1')
for tolerance in [0.1, 0.2, 0.3,]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/273_subdivide_bez_hatch.svg')

# postcard

In [ ]:
# make page
paper_size = '7x5 inches'
border:float=10
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
xgen = ss.uniform(loc=0.3, scale=0.02).rvs
split_func = functools.partial(random_bezier_subdivide, x0=0.3, x1=0.9, n_eval_points=20)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=3,
    buffer_frac=-0.0
)


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 2

inds = np.arange(len(bps))
split_bps = np.array_split(bps, n_layers)

layers = []
for ii in range(n_layers):
    this_bps = split_bps[ii]
    fills = []
    for p in this_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 1.2)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.6),
            degrees=np.random.uniform(-90,90),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify --tolerance 0.1')
for tolerance in [0.1, 0.2, 0.3,]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/275_subdivide_bez_hatch_postcard.svg')

# postcard

In [ ]:
# make page
paper_size = '14x11 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
xgen = ss.uniform(loc=0.3, scale=0.02).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=7,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 2

inds = np.arange(len(bps))
split_bps = np.array_split(bps, n_layers)

layers = []
for ii in range(n_layers):
    this_bps = split_bps[ii]
    fills = []
    for p in this_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 1.2)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.5),
            degrees=np.random.uniform(-90,90),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify --tolerance 0.1')
for tolerance in [0.1, 0.2, 0.3,0.6]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/code/side/plotter_images/oned_outputs/276_subdivide_bez_hatch.svg')

In [ ]:
# make page
paper_size = '6x6 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
circ = drawbox.centroid.buffer(55)

In [ ]:
xgen = ss.uniform(loc=0.3, scale=0.02).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    circ, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=7,
    buffer_frac=-0.0
)


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
for bp in bps:
    choice = np.random.choice(n_layers)
    layers[choice].append

In [ ]:
n_layers = 3

inds = np.arange(len(bps))
split_bps = np.random.choice(n_layers,size=len(bps))

layers = []
for ii in range(n_layers):
    this_bps = np.where(split_bps==ii)[0]
    fills = []
    for ind in this_bps:
        p = bps[ind]
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 1.2)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.4),
            degrees=np.random.uniform(-90,90),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify --tolerance 0.1')
for tolerance in [0.1, 0.2, 0.3,]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/277_subdivide_bez_hatch.svg')